In [1]:
import os
import warnings
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
warnings.filterwarnings('ignore')

os.environ["SERPER_API_KEY"]=SERPER_API_KEY
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [2]:
from crewai import Agent, Task, Crew
from crewai.process import Process

from crewai_tools import (
    SerperDevTool,
    WebsiteSearchTool,
    ScrapeWebsiteTool
)

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-mini")

search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrap_tool = ScrapeWebsiteTool()

researcher = Agent(
    role='테크 트랜드 연구원',
    goal="인공지능 분야의 최신 기술 트랜드를 한국어로 제공합니다. 지금은 2024년 12월입니다.",
    backstory="기술 트랜드에 예리한 안목을 지닌 전문 분석가이자 AI 개발자입니다.",
    tools=[search_tool, web_rag_tool],
    max_iter=5,
    llm=llm
)

writer = Agent(
    role='뉴스레터 작성자',
    goal="최신 AI 기술 트랜드에 대한 매력적인 테크 뉴스레터를 한국어로 작성하세요. 지금은 2024년 12월입니다.",
    backstory='기술에 대한 열정을 가진 숙련된 작가입니다.',
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [7]:
research = Task(
    description="AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요",
    expected_output="AI 업계에서 가장 주목받는 3대 기술 개발 동향과 그 중요성에 대한 신선한 관점을 요약한 글",
    agent=researcher
)

write = Task(
    description="""테크 트랜드 연구원의 요약을 바탕으로 AI 산업에 대한 매력적인 테크 뉴스레터를 작성하세요.
    테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다.""",
    expected_output="최신 기술 관련 소식을 재밌는 말투로 소개하는 4문단짜리 마크다운 형식 뉴스레터",
    agent=writer,
    output_file='C:\\Users\\semte\\Desktop\\rag\\new_post.md'
)

In [8]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    process=Process.sequential
)

In [ ]:
result = crew.kickoff()